In [21]:
import numpy as np
import json
import requests
import pandas as pd

url_cards = "https://db.ygoprodeck.com/api/v7/cardinfo.php"
url_sets = "https://db.ygoprodeck.com/api/v7/cardsets.php"

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

cards = requests.get(url_cards)
sets = requests.get(url_sets)

df_cards = pd.DataFrame.from_dict(cards.json()['data'])
df_sets = pd.DataFrame.from_dict(sets.json())

In [ ]:
#remove sets with missing date
df_sets = df_sets[df_sets['tcg_date'].notna()]

#removing cards in the database without an associated set
df_tcg_cards=df_cards[df_cards['card_sets'].notna()]

#aligning set names from the card pool
for i in df_tcg_cards['card_sets']:
    if i[0]['set_name'] == '2020 Tin of Lost Memories':
        i[0].update({'set_name' : '2020 Tin of Lost Memories Mega Pack'})

In [28]:
from datetime import datetime

#creating a date column to eventually add to the cards table
dates = []

for row in df_tcg_cards['card_sets']:
    dates_holder =[]
    for i in range(len(row)):
        
        #below conditional excludes set names that exist in the cards table, but are not in the sets table
        if len(df_sets.loc[df_sets['set_name']==row[i]['set_name'],'tcg_date']) > 0:
            date_str = df_sets.loc[df_sets['set_name']==row[i]['set_name'],'tcg_date'].item()
        
        #convert dates from sets table into datetime
        date = datetime.strptime(date_str, '%Y-%m-%d')
        dates_holder.append(date)
        
    #filter to only select the date of the cards' debuts
    date_earliest = min(dates_holder)
    dates.append(date_earliest)

#Adding dates to the cards table in a new column "release_date" 
df_tcg_cards['release_date'] = dates

In [32]:
df_tcg_cards

,id,name,type,desc,race,archetype,card_sets,card_images,card_prices,atk,def,level,attribute,banlist_info,scale,linkval,linkmarkers,release_date
0,34541863,"""A"" Cell Breeding Device",Spell Card,"During each of your Standby Phases, put 1 A-Co...",Continuous,Alien,"[{'set_name': 'Force of the Breaker', 'set_cod...","[{'id': 34541863, 'image_url': 'https://storag...","[{'cardmarket_price': '0.13', 'tcgplayer_price...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2007-05-16
1,64163367,"""A"" Cell Incubator",Spell Card,Each time an A-Counter(s) is removed from play...,Continuous,Alien,"[{'set_name': 'Gladiator's Assault', 'set_code...","[{'id': 64163367, 'image_url': 'https://storag...","[{'cardmarket_price': '0.11', 'tcgplayer_price...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2007-11-14
2,91231901,"""A"" Cell Recombination Device",Spell Card,Target 1 face-up monster on the field; send 1 ...,Quick-Play,Alien,"[{'set_name': 'Invasion: Vengeance', 'set_code...","[{'id': 91231901, 'image_url': 'https://storag...","[{'cardmarket_price': '0.12', 'tcgplayer_price...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-11-03
3,73262676,"""A"" Cell Scatter Burst",Spell Card,"Select 1 face-up ""Alien"" monster you control. ...",Quick-Play,Alien,"[{'set_name': 'Strike of Neos', 'set_code': 'S...","[{'id': 73262676, 'image_url': 'https://storag...","[{'cardmarket_price': '0.09', 'tcgplayer_price...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2007-02-28
4,37478723,"""Infernoble Arms - Durendal""",Spell Card,While this card is equipped to a monster: You ...,Equip,Noble Knight,"[{'set_name': 'Rise of the Duelist', 'set_code...","[{'id': 37478723, 'image_url': 'https://storag...","[{'cardmarket_price': '7.36', 'tcgplayer_price...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-08-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11476,87008374,ZW - Phoenix Bow,Effect Monster,"You can target 1 face-up ""Number C39: Utopia R...",Winged Beast,Utopia,"[{'set_name': 'Return of the Duelist', 'set_co...","[{'id': 87008374, 'image_url': 'https://storag...","[{'cardmarket_price': '0.13', 'tcgplayer_price...",1100.0,0.0,3.0,FIRE,NaN,NaN,NaN,NaN,2012-08-24
11477,2648201,ZW - Sleipnir Mail,Effect Monster,"You can target 1 ""Utopia"" monster you control;...",Beast,Utopia,"[{'set_name': 'Primal Origin', 'set_code': 'PR...","[{'id': 2648201, 'image_url': 'https://storage...","[{'cardmarket_price': '0.11', 'tcgplayer_price...",1000.0,1000.0,4.0,LIGHT,NaN,NaN,NaN,NaN,2014-05-16
11479,81471108,ZW - Tornado Bringer,Effect Monster,"You can target 1 ""Utopia"" monster you control;...",Dragon,Utopia,"[{'set_name': 'Cosmo Blazer', 'set_code': 'CBL...","[{'id': 81471108, 'image_url': 'https://storag...","[{'cardmarket_price': '0.06', 'tcgplayer_price...",1300.0,1800.0,5.0,WIND,NaN,NaN,NaN,NaN,2013-01-25
11480,18865703,ZW - Ultimate Shield,Effect Monster,When this card is Normal or Special Summoned: ...,Aqua,Utopia,"[{'set_name': 'Cosmo Blazer', 'set_code': 'CBL...","[{'id': 18865703, 'image_url': 'https://storag...","[{'cardmarket_price': '0.03', 'tcgplayer_price...",0.0,2000.0,4.0,EARTH,NaN,NaN,NaN,NaN,2013-01-25
